In [ ]:
!pip install mlxtend


In [1]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
path = 'D:/Disco/Data/PUCRS/proj CDIA III/'

tables = ['PUC_VENDAS', 'PUC_CAMPANHAS']

In [3]:
vendas_dataset = pq.ParquetDataset(f'{path}/dataset/{tables[0]}')
#campanhas_dataset = pq.ParquetDataset(f'{path}/dataset/{tables[1]}')

In [4]:
vendas = vendas_dataset.read_pandas()
#campanhas = campanhas_dataset.read_pandas()

In [5]:
vendas = vendas.to_pandas()


In [39]:
products = produtos = vendas.groupby(['COD_SKU', 'SKU', 'CATEGORIA_SKU', 'SUBCATEGORIA_SKU'], as_index=False).agg({'PRECO_REGULAR': 'mean'})
products

,COD_SKU,SKU,CATEGORIA_SKU,SUBCATEGORIA_SKU,PRECO_REGULAR
0,7,SERVICO EM DOMICILIO,DIVERSOS,SERVICOS TELE ENTREGA,5.000000
1,10,MANIPULADOS,MANIPULADOS,MANIPULADOS MEDICAMENTOS,0.010000
2,11,SERVICO DE APLICACAO DE INJETAVEIS,SERVIÇOS,SERVICOS FARMACEUTICOS,5.000000
3,12,SERVICO DE GESTO VACINAL GRIPE CONVENIOS,SERVIÇOS,SERVICOS FARMACEUTICOS,7.983931
4,13,SERVICO DE VERIFICACAO DE GLICEMIA CAPIL,SERVIÇOS,SERVICOS FARMACEUTICOS,3.000000
...,...,...,...,...,...
20653,100027853,ESC CAB MARCO BONI 7,PERFUMARIA,PERFUMARIA,14.990000
20654,100027855,ESC CAB MARCO BONI 8,PERFUMARIA,PERFUMARIA,29.990000
20655,100027878,NEBULIZADOR PULMOMAI,MEDICAMENTOS,LIBERADOS,219.900000
20656,100027882,AMPOLA PANTENE 3UN NUTRE,PERFUMARIA,PERFUMARIA,27.956292


In [36]:
len(products['COD_SKU'].unique())

20658

In [ ]:
vendas.head()

In [59]:
def extract_rules(transactions_df, min_support, min_threshold, metric):
    
    # Assuming 'transaction_data' is a list of lists where each sublist contains SKUs bought in a transaction.
    te = TransactionEncoder()
    te_ary = te.fit(transactions_df).transform(transactions_df)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    # Apply the apriori algorithm with a minimum support of 0.01 (adjustable)
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)

    # Generate association rules with a minimum confidence of 0.5 (adjustable)
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)

    del te, te_ary, df_encoded, frequent_itemsets

    return rules

def get_suggestions(sku_list, rules_df):
    
    if not isinstance(rules_df['antecedents'].iloc[0], (list, tuple, set)):
        rules_df['antecedents'] = rules_df['antecedents'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    filtered_rules = rules_df[rules_df['antecedents'].apply(lambda x: any(sku in x for sku in sku_list))]
    
    sorted_rules = filtered_rules.sort_values(by='confidence', ascending=False)

    # Obter todos os SKUs sugeridos da coluna 'consequents'
    suggested_skus = sorted_rules['consequents'].explode().unique()
    
    # Selecionar no máximo 6 SKUs
    suggested_skus = suggested_skus[:6]
    
    return suggested_skus

In [ ]:
#sales_tmp = vendas[vendas['DATA_CUPOM'].dt.month == 2]

for month in vendas['DATA_CUPOM'].dt.month.unique():
    print(month, len(vendas[vendas['DATA_CUPOM'].dt.month == month]))

In [29]:
vendas[vendas['COD_CLIENTE'] > 0].groupby(by=['COD_CLIENTE'])['COD_SKU'].apply(list)

COD_CLIENTE
120.0                                        [91659, 100018365]
250.0         [10094645, 13427, 13428, 10039151, 10039158, 1...
253.0            [10031253, 10023183, 10027980, 2341, 10023764]
408.0         [10020092, 10020092, 10021788, 100012109, 1000...
460.0                                [100014271, 92578, 102085]
                                    ...                        
94288844.0                      [10001406, 100027739, 10016217]
94288852.0                                         [6073, 3381]
94288893.0                                 [10023226, 10023226]
94288950.0                                 [10001416, 10090997]
94289015.0                                 [10029528, 10022967]
Name: COD_SKU, Length: 782985, dtype: object

In [7]:
transaction_data = vendas[vendas['COD_CLIENTE'] > 0].groupby(by=['COD_CLIENTE'])['COD_SKU'].apply(list)

In [8]:
metric = "confidence"
min_support=0.003
min_threshold = 0.03

# Convert the data into a transactional format where each transaction (COD_CUPOM) lists the products (COD_SKU) purchased
# Grouping the data by 'COD_CUPOM' and aggregating the SKUs purchased in each transaction
# sales_tmp.groupby('COD_CUPOM')['COD_SKU'].apply(list)
sample_data = transaction_data.sample(frac=.25)
rules = extract_rules(sample_data, min_support=min_support, min_threshold=min_threshold, metric=metric)

In [66]:
rules.to_csv('rules.csv')

In [9]:
rules
#.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(32),(10038480),0.004920,0.006498,0.004909,0.997923,153.568763,0.004877,478.371108,0.998400
1,(10038480),(32),0.006498,0.004920,0.004909,0.755503,153.568763,0.004877,4.069911,0.999986
2,(6833),(10032109),0.029952,0.050050,0.003101,0.103531,2.068562,0.001602,1.059657,0.532523
3,(10032109),(6833),0.050050,0.029952,0.003101,0.061958,2.068562,0.001602,1.034120,0.543789
4,(11864),(10027058),0.027352,0.038703,0.003116,0.113934,2.943774,0.002058,1.084904,0.678868
...,...,...,...,...,...,...,...,...,...,...
269,"(10035299, 10035311)","(10035289, 10035291)",0.007515,0.008302,0.004189,0.557444,67.149180,0.004127,2.240842,0.992567
270,(10035289),"(10035299, 10035291, 10035311)",0.015832,0.005446,0.004189,0.264601,48.587882,0.004103,1.352402,0.995174
271,(10035291),"(10035289, 10035299, 10035311)",0.014422,0.005548,0.004189,0.290471,52.355950,0.004109,1.401567,0.995253
272,(10035299),"(10035289, 10035291, 10035311)",0.015019,0.005257,0.004189,0.278912,53.057165,0.004110,1.379502,0.996113


In [65]:
get_suggestions([10038480, 32, 10032109, 6833, 10027058], rules)

array([10038480, 32, 10035289, 10035299, 10032109, 10027058], dtype=object)

In [17]:
vendas[vendas['COD_SKU'].isin([10038480, 32, 10032109, 6833, 10027058])]

,COD_CUPOM,COD_CLIENTE,CLIENTE_FISICO_JURIDICO,SEXO_CLIENTE,DTNASCIMENTO_CLIENTE,COD_SKU,SKU,CATEGORIA_SKU,SUBCATEGORIA_SKU,UF_CIDADE,COD_CIDADE,NOME_CIDADE,DATA_CUPOM,UNIDADES,IDENTIFICADOR_PROMOCIONAL,PRECO_REGULAR,TOTAL_DESCONTO,TOTAL_BRUTO,TOTAL_LIQUIDO,COD_LOJA
281,157222,34233277.0,Pessoa Física,F,1984-07-04,10027058,PAPEL HIG STYLUS FOL,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-04-15 18:02:12.000,1.0,NaN,12.99,0.00,12.99,12.99,1001
282,155066,34233277.0,Pessoa Física,F,1984-07-04,10027058,PAPEL HIG STYLUS FOL,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-04-03 18:07:15.000,1.0,NaN,12.99,0.00,12.99,12.99,1001
360,170653,34119812.0,Pessoa Física,M,1946-03-09,6833,SORINAN AD 30ML PHARMASCIENCE,MEDICAMENTOS,SIMILAR ONEROSO,RS,80400,PORTO ALEGRE,2024-06-28 14:26:52.350,1.0,NaN,8.49,0.00,8.49,8.49,1001
391,165084,34005888.0,Pessoa Física,F,1970-02-26,10032109,PAPEL HIG LOUVRE FOLHA DUPLA 20M 12UN NE,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-05-31 12:27:48.320,1.0,100336439.0,13.99,4.09,13.99,9.90,1001
655,149857,37957844.0,Pessoa Física,M,1977-08-07,6833,SORINAN AD 30ML PHARMASCIENCE,MEDICAMENTOS,SIMILAR ONEROSO,RS,80400,PORTO ALEGRE,2024-03-06 13:29:30.000,1.0,NaN,7.99,1.60,7.99,6.39,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9270549,520377,41713703.0,Pessoa Física,M,1963-12-16,10027058,PAPEL HIG STYLUS FOL,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-06-15 13:27:38.227,1.0,NaN,12.99,0.00,12.99,12.99,988
9270817,494580,79106155.0,Pessoa Física,M,1996-08-12,10032109,PAPEL HIG LOUVRE FOLHA DUPLA 20M 12UN NE,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-05-06 13:29:27.409,1.0,100333647.0,13.99,4.09,13.99,9.90,988
9270892,447994,41725107.0,Pessoa Física,M,1928-09-18,10032109,PAPEL HIG LOUVRE FOLHA DUPLA 20M 12UN NE,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-01-31 21:01:34.000,2.0,100283945.0,11.99,4.18,23.98,19.80,988
9271004,497436,41755120.0,Pessoa Física,F,1982-05-08,10032109,PAPEL HIG LOUVRE FOLHA DUPLA 20M 12UN NE,PERFUMARIA,HIGIENE PESSOAL,RS,80400,PORTO ALEGRE,2024-05-09 17:32:19.066,2.0,100333930.0,13.99,8.18,27.98,19.80,988


In [ ]:
#frequent_itemsets 
# Sort frequent itemsets by support in descending order
sorted_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# Select the top 20 most frequent items
top_20_items = sorted_itemsets.head(20)

print("Top 20 most frequent items:")
print(top_20_items)

# If you want to visualize these items
plt.figure(figsize=(12, 6))
plt.bar(range(len(top_20_items)), top_20_items['support'])
plt.xticks(range(len(top_20_items)), top_20_items['itemsets'].apply(lambda x: ', '.join(list(x))), rotation=90)
plt.xlabel('Items')
plt.ylabel('Support')
plt.title('Top 20 Most Frequent Items')
plt.tight_layout()
plt.show()

# Select items with support greater than 1%
relevant_items = frequent_itemsets[frequent_itemsets['support'] > 0.01]

print("\nItems with support greater than 1%:")
print(relevant_items)


In [ ]:
df = vendas

# Creating a pivot table where rows are products (SKU), columns are products, and values are the count of co-occurrences
product_matrix = df.groupby(['COD_CUPOM', 'CATEGORIA_SKU'])['UNIDADES'].sum().unstack().fillna(0)

# Creating a correlation matrix to find relationships between products
product_correlation = product_matrix.corr()

# Plotting the heatmap to show relationships between products
plt.figure(figsize=(10, 8))
sns.heatmap(product_correlation, cmap='coolwarm', annot=False)
plt.title('Correlação dos produtos')
plt.show()